# IS362 Final Project
## Data analysis of the US Energy Information Administration (EIA)
### Anthony Paveglio
---
The EIA is a federal agency associated witht he US Department of Energy. The EIA is responsible for collecting data regarding various forms of energy in the US including fossil fuels, renewable energy, and a global outlook on energy and its impact. The usage, production, and transportation of forms of energy are tracked through the EIA.

- Fossil Fuel: A natural form of energy such as coal, gas, or oil.
- Renewable Energy: Also a natural form of energy collected from sources quicker to replenish such as solar, wind, or hydroelectric.

_More info will appear here, in progress..._

---

# 1. Libraries and Imported files

In [30]:
#Necessary libraries and imports
import sys
import requests
import json
import pandas
import seaborn
import numpy
import datetime

# 2. API Access Request and Handling
## Making a connection with US EIA data sources.

In [31]:
categoryData = requests.get(
    "http://api.eia.gov/category/?api_key=ba1598e22a90a4bb48e3c5bdc55db041&category_id=371&out=json")

print 'Response code from EIA API:', categoryData.status_code, categoryData.reason
print 'Amount of time required for request:', categoryData.elapsed

Response code from EIA API: 200 OK
Amount of time required for request: 0:00:00.148534


## Functions to convert EIA data to usable Python data

In [32]:
def JSONtoDataFrameEIA(APIResponse, columnTitles=None):
    #Takes the returned data from the API call and converts it into a usable dataframe
    #EIA responses contain the actual data and a list of headers embedded in JSON
    #Also converts all string date data to a usable datetime
    #The EIA structures their dates as 'yyyymm'
    #Example: 202001 = 2020/1 = January 2020

    #Parameters:
    #    APIResponse (mixed): The API response to be parsed into a DataFrame.
    #    columnTitles (list, optional): List of column titles to apply over the Dataframe's columns
    #         if the columnTitles are not specified in the function call, default titles will be
    #         assigned based on the data within the returned API call data

    #Returns:
    #    EIADataFrame(DataFrame): EIA Data parsed into a Dataframe
    
    EIADataFrame = pandas.DataFrame(APIResponse.json()['series'][0]['data'])
    if columnTitles:
        EIADataFrame.columns = columnTitles
    else:
        EIADataFrame.columns = ['Date', APIResponse.json()['series'][0]['name']]
    EIADataFrame['Date'] = pandas.to_datetime(EIADataFrame['Date'], format='%Y%m')
    return EIADataFrame

## Functions for handling other misc. processes

In [33]:
#ToDo

# 3. Energy Usage in the United States
## Reviewing Fossil and Renewable sources
Lets compare the usage of both fossil fuels and renewable energy consumption in the United States. First we need to request the data series from the EIA via the API. This will return the latest dataset available.

In [34]:
fossilFuelGeneric = requests.get(
    "http://api.eia.gov/series/?api_key=ba1598e22a90a4bb48e3c5bdc55db041&series_id=TOTAL.FFTCBUS.M")
renewablesGeneric = requests.get(
    "http://api.eia.gov/series/?api_key=ba1598e22a90a4bb48e3c5bdc55db041&series_id=TOTAL.RETCBUS.M")

The data returned from the EIA is contained in a JSON format. In Section 2 of this Jupyter Notebook a function was created to handle parsing data from the EIA JSON format to retrieve the data from JSON and form it into a Jupyter Notebook. Below the EIA data will be formed into two sets of data titled _fossilFuelData_ and _renewablesData_. These DataFrames will contain the dates and units of fuel consumed for each recorded date

In [35]:
fossilFuelData = JSONtoDataFrameEIA(fossilFuelGeneric)
renewablesData = JSONtoDataFrameEIA(renewablesGeneric)

Merging these dataframes using _pandas.merge_ will combine the data found in both DataFrames into one which will make it easier to compare the amount of renewable versus fossil fuels consumed. Merging dataframes by specifying the _on_ keyword in the merge function will combine the data based on the date.

In [36]:
fuelData = pandas.merge(left=fossilFuelData, right=renewablesData, on='Date')

#This will display the last 25 months of recorded data for both Fossil Fuel and Renewable Energy consumption
display(fuelData.head(25))

,Date,"Total Fossil Fuels Consumption, Monthly","Total Renewable Energy Consumption, Monthly"
0,2020-01-01,7204.205,972.738
1,2019-12-01,7174.573,947.972
2,2019-11-01,6764.463,910.772
3,2019-10-01,6379.894,922.997
4,2019-09-01,6272.004,883.759
5,2019-08-01,6849.707,934.983
6,2019-07-01,6820.898,975.007
7,2019-06-01,6174.786,996.219
8,2019-05-01,6136.590,1059.511
9,2019-04-01,5958.825,1023.090


In [37]:
pivotFuelData = fuelData[fuelData['Date'] < '2020'].pivot_table(
    values=["Total Fossil Fuels Consumption, Monthly",
            "Total Renewable Energy Consumption, Monthly"],
    index=fuelData["Date"].dt.year,
    aggfunc=numpy.sum)

display(pivotFuelData)

,"Total Fossil Fuels Consumption, Monthly","Total Renewable Energy Consumption, Monthly"
Date,,
1973,70282.079,4410.937
1974,67871.940,4741.852
1975,65323.405,4687.121
1976,69070.873,4727.150
1977,70950.267,4208.967
1978,71809.332,5005.441
1979,72844.236,5122.854
1980,69779.558,5428.342
1981,67518.799,5413.688
